In [ ]:
import pandas as pd
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, Bidirectional, GRU 
from keras import Model
from keras import optimizers
from tensorflow import keras

In [ ]:
data_full = pd.read_csv('Hourly Volatility Dataset.csv')

In [ ]:
X = data_full[["Return_Squared", "Hourly Volatility"]]
Y = data_full["target"]
data_set = data_full[["Date","Return_Squared", "Hourly Volatility", "target"]]

In [ ]:
#Train-test split
splitlimit = int(len(data_full)*0.8)
training_features, test_data = data_set[:splitlimit], data_set[splitlimit:]

In [ ]:
#Outlier Detection in training_data_features

training_features["hourly_volatility_rolling_median"] = training_features["Hourly Volatility"].rolling(window=41, center=True, min_periods=1).median()
training_features["return_squared_rolling_median"] = training_features["Return_Squared"].rolling(window=41, center=True, min_periods=1).median()
training_features["volatility minus median"] = (training_features["Hourly Volatility"] - training_features["hourly_volatility_rolling_median"]).abs()
training_features["return minus median"] = (training_features["Return_Squared"] - training_features["return_squared_rolling_median"]).abs()
volatility_outliers_removed = training_features[~(training_features['volatility minus median'] > 5 * training_features['volatility minus median'].median())]
both_outliers_removed = volatility_outliers_removed[~(volatility_outliers_removed['return minus median'] > 5 * volatility_outliers_removed['return minus median'].median())]

In [ ]:
X_cleaned = both_outliers_removed[["Return_Squared", "Hourly Volatility"]]
Y_cleaned = both_outliers_removed["target"]
data_set_cleaned = both_outliers_removed[["Return_Squared", "Hourly Volatility", "target"]]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
training_data_features_scaled = scaler.fit_transform(X_cleaned)
data_set_scaled = scaler.fit_transform(data_set_cleaned)

In [ ]:
#Reconstructing training data 

Z = []

backcandles = 15

for j in range(2):
    Z.append([])
    for i in range(backcandles, training_data_features_scaled.shape[0]):
        Z[j].append(training_data_features_scaled[i-backcandles:i, j])
        
Z = np.moveaxis(Z, [0], [2])
Z, yi = np.array(Z), np.array(data_set_scaled[backcandles-1:, -1])
y_final = np.reshape(yi,(len(yi),1))
y_final = y_final[:-1]

In [ ]:
#OGRU Cell and Layer
import tensorflow as tf
from tensorflow.keras.layers import Layer

           
class OGRUCell(Layer):
    def __init__(self, units, **kwargs):
        super(OGRUCell, self).__init__(**kwargs)
        self.units = units
        self.state_size = units  
        self.output_size = units  
        
    def build(self, input_shape):
        self.Wr = self.add_weight(shape=(input_shape[-1], self.units),
                                  initializer='glorot_uniform',
                                  name='Wr')
        self.Ur = self.add_weight(shape=(self.units, self.units),
                                  initializer='orthogonal',
                                  name='Ur')
        self.Wz = self.add_weight(shape=(input_shape[-1], self.units),
                                  initializer='glorot_uniform',
                                  name='Wz')
        self.Uz = self.add_weight(shape=(self.units, self.units),
                                  initializer='orthogonal',
                                  name='Uz')
        self.W = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='glorot_uniform',
                                 name='W')
        self.U = self.add_weight(shape=(self.units, self.units),
                                 initializer='orthogonal',
                                 name='U')
        self.bias_r = self.add_weight(shape=(self.units,),
                                      initializer='zeros',
                                      name='bias_r')
        self.bias_z = self.add_weight(shape=(self.units,),
                                      initializer='zeros',
                                      name='bias_z')
        self.bias_h = self.add_weight(shape=(self.units,),
                                      initializer='zeros',
                                      name='bias_h')

    
    def call(self, inputs, states):
        prev_h = states[0]

        r = tf.sigmoid(tf.matmul(inputs, self.Wr) + tf.matmul(prev_h, self.Ur) + self.bias_r)
        
        r_reduced = tf.reduce_mean(r, axis=-1, keepdims=True)  # Averaging across the units dimension
        r_reduced = tf.tile(r_reduced, [1, inputs.shape[-1]])

      
        z = tf.sigmoid(tf.matmul(inputs * r_reduced, self.Wz) + tf.matmul(prev_h, self.Uz) + self.bias_z)

        n = tf.tanh(tf.matmul(inputs, self.W) + tf.matmul(prev_h * r, self.U) + self.bias_h)

        h = (1 - z) * prev_h + z * n

        return h, [h]


# Producing OGRU layer 
class OGRU(tf.keras.layers.RNN):
    def __init__(self, units, **kwargs):
        self.cell = OGRUCell(units, **kwargs)
        super(OGRU, self).__init__(self.cell, **kwargs)

# OGRU layer with 80 units
ogru_layer = OGRU(units=80)

In [ ]:
#GRU model
gru_input = Input(shape = (backcandles, 2), name = 'gru_input')

inputs = GRU(80, name='first_layer')(gru_input)

inputs = Dense(1, name='dense_layer')(inputs)

output = Activation('sigmoid', name = 'output')(inputs)

model = Model(inputs = gru_input, outputs = output)

model.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ["accuracy"])

history = model.fit(x=Z, y=y_final, epochs = 30, validation_data = (Z, y_final))

In [ ]:
#OGRU model
ogru_input = Input(shape = (backcandles, 2), name = 'ogru_input')

inputs = ogru_layer(ogru_input)

inputs = Dense(1, name='dense_layer')(inputs)

output = Activation('sigmoid', name = 'output')(inputs)

model2 = Model(inputs = ogru_input, outputs = output)

model2.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ["accuracy"])

history2 = model2.fit(x=Z, y=y_final, epochs = 30, validation_data = (Z, y_final))

In [ ]:
#gru plot
history.history.keys() 
plt.plot(history.history['loss'], color = 'red', label = "Training Loss")
plt.plot(history.history['accuracy'], color = 'royalblue', label = "Training Accuracy")
plt.xlabel('Number of Epochs', fontsize = 15)
plt.ylabel('Accuracy', fontsize = 15)
plt.gca().tick_params(axis='x', labelsize=12)
plt.gca().tick_params(axis='y', labelsize=12)
plt.legend(fontsize= 12)
#plt.savefig('GRU.png', format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#ogru plot
history2.history.keys() 
plt.plot(history3.history['loss'], color = 'red', label = "Training Loss")
plt.plot(history3.history['accuracy'], color = 'royalblue', label = "Training Accuracy")
plt.xlabel('Number of Epochs', fontsize = 15)
plt.ylabel('Accuracy', fontsize = 15)
plt.gca().tick_params(axis='x', labelsize=12)
plt.gca().tick_params(axis='y', labelsize=12)
plt.legend(fontsize= 12)
#plt.savefig('MyGRU.png', format='png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
X_test = test_data[["Return_Squared", "Hourly Volatility"]]
Y_test = test_data["target"]
test_dataset = test_data[["Return_Squared", "Hourly Volatility", "target"]]

In [ ]:
test_scaled = scaler.fit_transform(test_dataset)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
T = []

backcandles = 10

for j in range(2):
    T.append([])
    for i in range(backcandles, X_test_scaled.shape[0]):
        T[j].append(X_test_scaled[i-backcandles:i, j])

In [ ]:
T = np.moveaxis(T, [0], [2])
T, yi_test = np.array(T), np.array(test_scaled[backcandles-1:, -1])
y_final_test = np.reshape(yi_test,(len(yi_test),1))
y_final_test = y_final_test[:-1]

In [ ]:
#gru
test_predictions1 = model.predict(T)
test_predicted_classes1 = (test_predictions1 > 0.5).astype(int)
y_frame1 = test_data[["Date", "target"]].tail(14393- backcandles +1)
y_frame1 = y_frame1.iloc[:-1]
y_frame1['predicted'] = test_predicted_classes1
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_frame1['predicted'], y_frame1['target'])
print(cm)


In [ ]:
#ogru
test_predictions2 = model3.predict(T)
test_predicted_classes2 = (test_predictions2 > 0.5).astype(int)
y_frame2 = test_data[["Date", "target"]].tail(14393- backcandles +1)
y_frame2 = y_frame2.iloc[:-1]
y_frame2['predicted'] = test_predicted_classes2
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_frame2['predicted'], y_frame2['target'])
print(cm)

In [ ]:
#Below is an additional section which shows the performance for each year in the test set 

In [ ]:
import pandas as pd


y_frame['Date'] = pd.to_datetime(y_frame['Date'])
grouped_by_year = y_frame1.groupby(y_frame['Date'].dt.year)
dataframes_by_year = {year: group for year, group in grouped_by_year}
y_frame_2020 = dataframes_by_year[2020]
y_frame_2021 = dataframes_by_year[2021]
y_frame_2022 = dataframes_by_year[2022]
y_frame_2023 = dataframes_by_year[2023]
y_frame_2024 = dataframes_by_year[2024]

from sklearn.metrics import confusion_matrix
cm2020 = confusion_matrix(y_frame_2020['predicted'], y_frame_2020['target'])
cm2021 = confusion_matrix(y_frame_2021['predicted'], y_frame_2021['target'])
cm2022 = confusion_matrix(y_frame_2022['predicted'], y_frame_2022['target'])
cm2023 = confusion_matrix(y_frame_2023['predicted'], y_frame_2023['target'])
cm2024 = confusion_matrix(y_frame_2024['predicted'], y_frame_2024['target'])



In [ ]:
print(cm2024)

In [ ]:
print(cm2023)

In [ ]:
print(cm2022)

In [ ]:
print(cm2021)

In [ ]:
print(cm2020)

In [ ]:
import pandas as pd

# Assuming df is your DataFrame and 'date_column' is the name of your date column
y_frame['Date'] = pd.to_datetime(y_frame['Date'])
grouped_by_year = y_frame2.groupby(y_frame['Date'].dt.year)
dataframes_by_year = {year: group for year, group in grouped_by_year}
y_frame_2020 = dataframes_by_year[2020]
y_frame_2021 = dataframes_by_year[2021]
y_frame_2022 = dataframes_by_year[2022]
y_frame_2023 = dataframes_by_year[2023]
y_frame_2024 = dataframes_by_year[2024]

from sklearn.metrics import confusion_matrix
cm2020 = confusion_matrix(y_frame_2020['predicted'], y_frame_2020['target'])
cm2021 = confusion_matrix(y_frame_2021['predicted'], y_frame_2021['target'])
cm2022 = confusion_matrix(y_frame_2022['predicted'], y_frame_2022['target'])
cm2023 = confusion_matrix(y_frame_2023['predicted'], y_frame_2023['target'])
cm2024 = confusion_matrix(y_frame_2024['predicted'], y_frame_2024['target'])


In [ ]:
print(cm2024)

In [ ]:
print(cm2023)

In [ ]:
print(cm2022)

In [ ]:
print(cm2021)

In [ ]:
print(cm2020)